In [1]:
import mat73
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [2]:
data_dict_female = mat73.loadmat("./data/Emotrans1_girl_data_preprocessed.mat", use_attrdict=True)
arr_f = np.array(data_dict_female["All_Feature"])

In [3]:
data_dict = mat73.loadmat("./data/Emotrans1_Boy_data_preprocessed.mat", use_attrdict=True)
arr_m = np.array(data_dict["All_Feature"])

In [4]:
#retrive pos or neg return np 
def get_pos_or_neg(data,position):
    arr = []
    index = 0
    for i in range(data.shape[0]):
        temp = []
        for j in position:
            for k in data[i][j[0]][j[1]]:
                temp.append(k)
        arr.append(temp)
    return np.array(arr)

In [5]:
# select frequenceis and (0-4s -> 0 or 0.5-4.5s -> 1)
def get_selected(data=None,freq = None, sec=None):
    if freq:
        data = np.delete(data,freq,axis=3)
    if sec == 0:
        return np.delete(data, 0, axis=4)
    elif sec == 1:
        return np.delete(data,1,axis=4)
    else:
        print('sec not specified')
        return data 

In [6]:
# female data combined 
pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]]

pos_df_f = get_pos_or_neg(arr_f,pos)
neg_df_f = get_pos_or_neg(arr_f,neg)

pos_df_f=get_selected(pos_df_f,sec=0).reshape(15,96,128,8)
neg_df_f = get_selected(neg_df_f,sec=0).reshape(15,96,128,8)
raw_data_f = np.concatenate((pos_df_f,neg_df_f),axis=1).reshape(15,192,128*8)
y_f = np.concatenate((np.ones((15,96)),np.zeros((15,96))),axis=1)


In [10]:
pos_df_m = get_pos_or_neg(arr_m,pos)
neg_df_m = get_pos_or_neg(arr_m,neg)
pos_df_m = get_selected(pos_df_m,sec=0).reshape(18,96,128,8)
neg_df_m = get_selected(neg_df_m,sec=0).reshape(18,96,128,8)
raw_data_m = np.concatenate((pos_df_m,neg_df_m),axis=1).reshape(18,192,128*8)
y_m = np.concatenate((np.ones((18,96)),np.zeros((18,96))),axis=1)


In [19]:
raw_data = np.append(raw_data_f,raw_data_m,axis=0)
y = np.append(y_f,y_m,axis=0)
print(raw_data.shape)
print(y.shape)
# print(y.shape)

(33, 192, 1024)
(33, 192)


In [ ]:
iter = 0
alphas = [0.1,0.01,0.001]
coefs = []
for a in alphas:
    # ,max_iter=int(1e6)
    model = LogisticRegression(C=a, penalty='l1',max_iter=500,solver='saga')
    iter = 0
    train_scores=[]
    test_scores = []
    for i in range(32):
        display(iter)
        iter+=1
        x_train,x_test = np.concatenate((raw_data[:i],raw_data[i+1:])).reshape(32*192,1024),raw_data[i].reshape(192,1024)
        y_train,y_test = np.concatenate((y[:i],y[i+1:])).reshape(32*192),y[i].reshape(192)
        x_train,y_train = shuffle(x_train,y_train)
        model.fit(x_train, y_train)
        # training loss
        pred_train_lasso= model.predict(x_train)
        temp = accuracy_score(y_train,pred_train_lasso)
        display("training accuracy: ",temp)
        train_scores.append(temp)

        #testing 
        pred_test_lasso= model.predict(x_test)
        temp = accuracy_score(y_test,pred_test_lasso)
        display("testing accuracy: ",temp)
        test_scores.append(temp)
        
        coefs = model.coef_
    with open('comb_with_acc'+str(a)+".txt",'w') as f:
        f.write("accuracy - training")
        f.write(str(train_scores))
        f.write("\n accuracy - testing")
        f.write(str(test_scores))

        f.write("\ncoefs ")
        for c in coefs[0]:
            f.write(str(c)+",")
    print(train_scores)
    print(test_scores)
    print(coefs)

0

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


'training accuracy: '

0.61474609375

'testing accuracy: '

...

'training accuracy: '

0.5211588541666666

'testing accuracy: '

0.5052083333333334

22